In [27]:
## Actividad Integradora
 
#Estefanía Pérez Yeo - A01639270
#Vanessa Méndez Palacios - A01639925
#Salvador Fernando Camacho Hernández - A01634777
#Paola Elizabeth Gómez Martínez - A01633772
#Héctor Robles Villarreal - A01634105


In [28]:
import agentpy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random
from time import time

In [29]:
parameters = {
    'm': 20, # Altura del grid
    'n': 20, # Longitud del grid
    'k': 2, # Número de carros
    'tMax': 100 # Tiempo máximo de ejecucción
}
class IntersectionModel(ap.Model):
    
    def setup(self):
        # Generate random starting positions
        # Generar posiciones de inicio aleatorias
        random.seed(time())
        self.pos=[(1, round(self.p.n / 2) - 1), (self.p.m - 2, round(self.p.n / 2)), (round(self.p.m / 2), 1), (round(self.p.m / 2)-1, self.p.n - 2)]
        for i in range(0,self.p.k):
            self.pos.append(self.pos[random.randint(0,3)])
        # Crear agentes
        cars = self.agents = ap.AgentList(self, self.p.k)
        # Crear grid
        self.street = ap.Grid(self, (self.p.m, self.p.n), track_empty=True)
        self.street.add_agents(cars, positions=self.pos[4:14], empty=True)
    
        self.agents.condition = 0
        self.agents.orientation = 'x'
        
    def step(self):
        for i in self.agents:
            
            if(self.street.positions[i] == self.pos[0]):
                i.orientation = 'n' 
            if(self.street.positions[i] == self.pos[1]):
                i.orientation = 's'
            if(self.street.positions[i] == self.pos[2]):
                i.orientation = 'w'
            if(self.street.positions[i] == self.pos[3]):
                i.orientation = 'e'
                
            if(self.street.positions[i] == (self.p.m-2, round(self.p.n/2)-1) # Final 'n' position
              or self.street.positions[i] == (1, round(self.p.n/2)) # Final 's' position
              or self.street.positions[i] == (round(self.p.m/2), self.p.n-2) # Final 'w' position
              or self.street.positions[i] == (round(self.p.m/2)-1, 1)): # Final 'e' position
                i.orientation = 'x'
                self.street.move_to(i, self.pos[random.randint(0,3)])
                
            if(i.orientation == 'n' 
                and not self.street.agents[self.street.positions[i][0]+1, self.street.positions[i][1]].condition == True
                and not self.street.agents[self.street.positions[i][0]+2, self.street.positions[i][1]].condition == True):
                    self.street.move_by(i, (1, 0))
            if(i.orientation == 's'
               and not self.street.agents[self.street.positions[i][0]-1, self.street.positions[i][1]].condition == True
               and not self.street.agents[self.street.positions[i][0]-2, self.street.positions[i][1]].condition == True):
                    self.street.move_by(i, (-1, 0))
            if(i.orientation == 'w'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+2].condition == True):
                self.street.move_by(i, (0, 1))
            if(i.orientation == 'e'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-2].condition == True):
                self.street.move_by(i, (0, -1))           
        if self.t == self.p.tMax:
            self.stop()

    def end(self):
        return


In [30]:
def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#1831E7', None:'#000000'}
    ap.gridplot(attr_grid, ax = ax, color_dict = color_dict, convert = True)
    ax.set_title(f"Car intersection simulation\n"
                 f"Time-step: {model.t}")
fig, ax = plt.subplots()
model = IntersectionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))